# Program Repair

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8" # this is needed to get rid of weird colab locale error
# if you are still running into issues, please restart the runtime to initialize a new environment

In [ ]:
# installing the accelerate library
!pip install accelerate

In [ ]:
# https://github.com/evalplus/evalplus
!pip install evalplus==0.2.0

In [ ]:
!wget https://raw.githubusercontent.com/uiuc-cs598lmz-s24/hw3/main/buggy_humaneval.jsonl

In [ ]:
import json

def grab_buggy_dataset():
    inference_dataset = []
    file = "buggy_humaneval.jsonl"
    with open(file, "r") as f:
        inference_dataset.extend([json.loads(x) for x in f.readlines()])
    print("Number of tasks: {}".format(len(inference_dataset)))
    return inference_dataset

buggy_humaneval = grab_buggy_dataset()
# feel free to play around the dataset for a bit

In [11]:
import torch

from torch.utils.data import DataLoader
from transformers import AdamW, AutoTokenizer, AutoModelForCausalLM, T5ForConditionalGeneration

def load_codegen_model():
    tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-2B-mono")
    model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-2B-mono", device_map='auto', torch_dtype=torch.float16)
    return model, tokenizer

def load_codet5_model():
    tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-base")
    model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base", device_map='auto')
    return model, tokenizer

codegen, codegen_tokenizer = load_codegen_model()
codet5, codet5_tokenizer = load_codet5_model()

In [ ]:
# https://github.com/evalplus/evalplus
!pip install evalplus==0.2.0

In [ ]:
# make the folder to save the results
!mkdir codegen_fix

In [ ]:
import os
from tqdm.auto import tqdm


def codegen_program_repair(buggy_program: str, buggy_line_index: int, model, tokenizer) -> str:
    # TODO: implement greedy solution using codegen.
    # note, you want to return a complete function here
    # note, there might be some post processing needed to remove irrelevant
    # tokens.
    # you can access the buggy line using 'buggy_program.split("\n")[buggy_line_index]'
    return buggy_program


def repair_humaneval_codegen(model, tokenizer, buggy_humaneval, workdir):
  for bug in tqdm(buggy_humaneval):
      name = bug['task_id'].replace("/", "_")
      buggy_code = bug['buggy_code']
      buggy_line_index = bug['buggy_line']

      fixed_code = codegen_program_repair(buggy_code, buggy_line_index, model, tokenizer)
      os.makedirs(os.path.join(workdir, name), exist_ok=True)
      with open(os.path.join(workdir, name, '0.py'), 'w') as f:
          f.write(fixed_code)


# generate the solutions produced by codegen
repair_humaneval_codegen(codegen, codegen_tokenizer, buggy_humaneval, "codegen_fix")

In [ ]:
# now place take a look at the solutions produced by codegen in the folder
# we will now evaluate the solution
# note you can passing in "--i-just-wanna-run" to this command to
# recompute the results IF and ONLY IF you have made some updates to each solution file :)
!yes Y | evalplus.evaluate --dataset humaneval --samples codegen_fix --i-just-wanna-run

In [95]:
# make the folder to save the results
!mkdir codet5_fix

In [ ]:
import os
from tqdm.auto import tqdm


def codet5_program_repair(buggy_program: str, buggy_line_index: int, model, tokenizer) -> str:
    # TODO: implement greedy solution using codet5.
    # note, you want to return a complete function here
    # note, there might be some post processing needed to remove irrelevant
    # tokens.
    # you can access the buggy line using 'buggy_program.split("\n")[buggy_line_index]'
    return buggy_program


def repair_humaneval_codet5(model, tokenizer, buggy_humaneval, workdir):
  for bug in tqdm(buggy_humaneval):
      name = bug['task_id'].replace("/", "_")
      buggy_code = bug['buggy_code']
      buggy_line_index = bug['buggy_line']

      fixed_code = codet5_program_repair(buggy_code, buggy_line_index, model, tokenizer)
      os.makedirs(os.path.join(workdir, name), exist_ok=True)
      with open(os.path.join(workdir, name, '0.py'), 'w') as f:
          f.write(fixed_code)


repair_humaneval_codet5(codet5, codet5_tokenizer, buggy_humaneval, "codet5_fix")

In [ ]:
# now place take a look at the solutions produced by codegen in the folder
# we will now evaluate the solution
# note you can passing in "--i-just-wanna-run" to this command to
# recompute the results IF and ONLY IF you have made some updates to each solution file :)
!yes Y | evalplus.evaluate --dataset humaneval --samples codet5_fix --i-just-wanna-run

In [90]:
from evalplus.data import get_human_eval_plus

def check_which_passed(workdir: str, dataset):
    with open(os.path.join(workdir, "eval_results.json"), "r") as f:
        results = json.loads(f.read())

    failed_humaneval = []
    failed_humaneval_plus = []

    for task_id in dataset.keys():
        total = results['eval'][task_id]['nfiles']
        humaneval_base = len([x for x in results['eval'][task_id]['base'] if x[0] == "success"]) / total
        humaneval_plus = len([x for x in results['eval'][task_id]['plus'] if x[0] == "success"]) / total

        if humaneval_base == 1:
            failed_humaneval.append(task_id)
        if humaneval_plus == 1:
            failed_humaneval_plus.append(task_id)

    return failed_humaneval, failed_humaneval_plus


In [91]:
# you can use this to check which problem the model fixed
success_humaneval, success_humaneval_plus = check_which_passed("codegen_fix", get_human_eval_plus())

In [100]:
success_humaneval_codet5, success_humaneval_plus_codet5 = check_which_passed("codet5_fix", get_human_eval_plus())

# Template-based Program Repair

In [98]:
# make the folder to save the results
!cp -r codet5_fix codet5_fix_template

In [ ]:
def template_codet5_program_repair(buggy_program: str, buggy_line_index: int, model, tokenizer) -> str:
    # TODO: implement greedy solution using codet5.
    # note, you want to return a complete function here
    # note, there might be some post processing needed to remove irrelevant
    # tokens.
    # you can access the buggy line using 'buggy_program.split("\n")[buggy_line_index]'
    return buggy_program


def repair_humaneval_codet5_template(model, tokenizer, buggy_humaneval, fixed_bugs, workdir):
  for bug in tqdm(buggy_humaneval):
      name = bug['task_id'].replace("/", "_")
      if bug['task_id'] in fixed_bugs:
        continue
      buggy_code = bug['buggy_code']
      buggy_line_index = bug['buggy_line']

      fixed_code = template_codet5_program_repair(buggy_code, buggy_line_index, model, tokenizer)
      os.makedirs(os.path.join(workdir, name), exist_ok=True)
      with open(os.path.join(workdir, name, '0.py'), 'w') as f:
          f.write(fixed_code)

# generate the solutions produced by codegen
repair_humaneval_codet5_template(codet5, codet5_tokenizer, buggy_humaneval, success_humaneval_codet5, "codet5_fix_template")

In [ ]:
# note you can passing in "--i-just-wanna-run" to this command to
# recompute the results IF and ONLY IF you have made some updates to each solution file :)
# you may need to pass in (yes Y | command) on colab
!yes Y | evalplus.evaluate --dataset humaneval --samples codet5_fix_template --i-just-wanna-run